# DRAFT - Managing the Db2 Data Management Console - DRAFT

This Jupyter Notebook contains examples of how to setup and manage the Db2 Data Management Console. It covers how to add additional users using database authentication, how to explore and manage connections and setup and manage monitoring profiles.

The Db2 Data Management Console is more than a graphical user interface. It is a set of microservices that you can use to build custom applications to automate your use of Db2.

This Jupyter Notebook contains examples of how to use the Open APIs and the composable interface that are available in the Db2 Data Management Console. Everything in the User Interface is also available through an open and fully documented RESTful Services API. The full set of APIs are documented as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to the documentation directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

You can also embed elements of the full user interface into an IFrame by constructing the appropriate URL.

This hands on lab will be calling the Db2 Data Management Console as a service. However you can explore it through the user interface as well. Just click on the following link to try out the console that is already and setup in this lab: http://localhost:11080/console. If you have not already logged in you can use the following:
* Userid: db2inst1
* Password: db2inst1

### Import Helper Classes
For more information on these classes, see the Lab on Automate Db2 with Open Console Services

### The Console Class
Next we will create a Console helper class that will encapsulate the Rest API calls that we can use to directly access the Db2 Data Management Console service without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

The full set of APIs are documents as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to that directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2Console():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
                return True;
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
                return True;
        else:
            print ('Unable to authenticate, no bearer token obtained')
            return False;
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def putRequest(self, api, json=None):
        return requests.put(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getUserPriviledges(self, profile=''):
        if profile == '' :
            return self.getRequest('/userProfilePrivileges')
        else : 
            return self.getRequest('/userProfilePrivileges/'+profile)
    
    def assignUserPrivileges(self, profile, user):
        json = [{'profileName': profile, 'USER':[user], 'OWNER':[]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
 
    def assignOwnerPrivileges(self, profile, owner):
        json = [{'profileName': profile, 'USER':[], 'OWNER':[owner]}]
        return self.postRequest('/userProfilePrivileges?action=assign', json) 
    
    def revokeProfilePrivileges(self, profile, user):
        json = [{'profileName': profile, 'USER':[user]}]
        return self.postRequest('/userProfilePrivileges?action=revoke', json) 

    def getProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    
    
    def getMonitorStatus(self):
        return self.getRequest('/monitor') 
    
    def getConsoleProfiles(self):
        return self.getRequest('/dbprofiles')  
    
    def getConsoleRepository(self):
        return self.getRequest('/repository')    

    def getExportConsoleProfiles(self):
        return self.getRequest('/dbprofiles/transfer/export?exportCred=true')
    
    def postConnectionProfile(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.postRequest('/dbprofiles', json)
    
    def putConnectionProfileUpdate(self, connectionName, dbName, port, host, userid, password, comment):
        json = {"name":connectionName,"location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","collectionCred":{"securityMechanism":"3","user":userid,"password":password},"operationCred":{"securityMechanism":"3","user":userid,"password":password,"saveOperationCred":"true"},"comment":comment}
        return self.putRequest('/dbprofiles/'+connectionName, json)
    
    def postTestConnection(self, dbName, port, host, userid, password):
        json = {"name":"","location":"","databaseName":dbName,"dataServerType":"DB2LUW","port":port,"host":host,"URL":"jdbc:db2://"+host+":"+port+"/"+dbName+":retrieveMessagesFromServerOnGetMessage=true;","sslConnection":"false","disableDataCollection":"false","operationCred":{"securityMechanism":"3","user":userid,"password":password}}
        return self.postRequest('/dbprofiles/testConnection', json)
    
    def deleteConnectionProfile(self, connectionName):
        return self.deleteRequest('/dbprofiles/'+connectionName)

    def getMonitoringProfiles(self):
        return self.getRequest('/monitorprofile/front')
    
    def getMonitoringProfile(self, profileID):
        return self.getRequest('/monitorprofile/front/'+profileID)
    
    def putMonitoringProfile(self, profileID, json):
        return self.putRequest('/monitorprofile/front/'+profileID, json)   
    
    def getProfileIndex(self, name):
        r = self.getMonitoringProfiles()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r) 
            profileList = pd.DataFrame(json_normalize(json['resources']))[['name','id']]
            profileList.set_index('name',inplace=True)
            index = profileList.loc[profileName][0]
            return(index)
        else:
            print(self.getStatusCode(r)) 

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [11]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
databaseAPI = Db2Console(Console+'/dbapi/v4')
if databaseAPI.authenticate(user, password) :
    print("Token Created")
else : 
    print("Token Creation Failed")
database = Console

Token Created


### Confirm the connection
To confirm that your connection is working you can the Console connection profiles.

In [4]:
# Get Console Connection Profiles
r = databaseAPI.getConsoleProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

,name,disableDataCollection,databaseVersion,databaseName,timeZone,databaseVersion_VRMF,sslConnection,userProfileRole,host,_PROFILE_INIT_,dataServerType,port,edition,dataServerExternalType,capabilities,location,tag
0,SAMPLE,false,11.5.0,SAMPLE,-50000,11.5.0.0,false,OWNER,localhost,true,DB2LUW,50000,"AESE,DEC",DB2LUW,"[""DSM_ENTERPRISE_LUW""]",,NaN
1,HISTORY,false,11.5.0,HISTORY,-50000,11.5.0.0,false,OWNER,localhost,true,DB2LUW,50000,"AESE,DEC",DB2LUW,"[""DSM_ENTERPRISE_LUW""]",,[HOT]


### Get Repository Configuration
You can also get details on the repository configuration. You can see that we are using a local database name HISTORY to store all the monitoring data collected by the console. 

In [5]:
# Get Console Repository Configuration
r = databaseAPI.getConsoleRepository()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    display(pd.DataFrame(json_normalize(json)[['databaseName','status','host','port','collectionCred.user']]).transpose())
else:
    print(databaseAPI.getStatusCode(r))      

,0
databaseName,history
status,active
host,localhost
port,50000
collectionCred.user,db2inst1


### Running SQL Routines
We need to run some SQL scripts later in the lab. So run the next two cells. They define routines that run SQL scripts and display the results of those scripts. If you want to learn more about how these routines were developed, check out the [Analyzing SQL Workloads notebook](http://localhost:8888/notebooks/Db2_Data_Management_Console_SQL.ipynb).

In [6]:
def runSQL(profile,user, password, sqlText):
    
    if databaseAPI.authenticate(user, password, profile) :

        runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        while 'results' not in json :
                json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
        fulljson = json

        while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
            json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
            while 'results' not in json :
                json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
            for results in json['results'] :
                fulljson['results'].append(results)
            time.sleep(1) 
        return fulljson
    else :
        print('Could not authenticate')
print('runSQL routine defined')

runSQL routine defined


In [7]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()
print('displayResults routine defined')

displayResults routine defined


## Setting up Repository Authentication

### Reviewing current console authentication and switching to repository authentication

Let's start by looking at the current authentication settings in the console. To see the authentication settings you can either:

Use the full console to switch to Repository Database Authentication:
1. Click the link http://localhost:11080/console
2. Click the gear icon at the top right of the page
3. Select **Repository Setting**

Use the embedded page from the console to switch to Repostory Database Authentication
1. Run the cell below

In [8]:
IFrame(database+'/console/?mode=compact#settings/authentication', width=1400, height=300)

To support multiple users you need a way to authenticate those other users. There are two choices. 
1. **Repository Database Authentication**. This delegates the authentication of console users to the Db2 repository database used by the console to store historical monitoring data. 
2. **LDAP**. This delegates the authentication to an external LDAP service.
In this example we use the Repository Database to authenticate console users. You associate console users and administrators to Db2 Authorities, Groups, UDFs or Role. 

In this lab we are going to setup the first option, Repository Database Authentication. 

Before switching to Repository Database Authentication lets create some new users in the operating system that Db2 can recognize. 

### Adding Operating System Users
The repository database uses standard Db2 authentiation. The simpiliest way to set this up is to use the userid authentication that is built into the operating system that your Db2 database is running on. 

To add new users to the console you need to add new users to your operating system. 

In this hands-on lab you can create those new users through the Linux settings console. ![Linux Settings](https://raw.githubusercontent.com/Db2-DTE-POC/db2dmc/master/media/LinuxSettings.png "Linux Settings")

1. Click the **Linux Settings** Icon
2. Click **Users** in the menu on the left side of the page
3. Click **Unlock** at the top right side of the page
4. Enter the DB2POC password **dmc2020pot**.
5. Click **Add User...** at the top-right of the page
6. Type **Peter** into the **Full Name** and the **Username** fields.
7. Click **Set a password now**
8. Enter **DataConsole** into the **password field**
9. Click **Add** at the top-right of the page

Repeat steps 5 to 9 for the following new users:
1. **Paul**
2. **Mary**

### Adding Users to the Console using Repository Authentication
Now that you have three new users lets add them grant them database authorities and CONSOLE roles. The following cell will grant a user to either an Admin or User role (CONSOLE_ADM, CONSOLE_USR). 

In [24]:
userList = {'newUser':['PETER', 'PAUL', 'MARY'], 'Type':['Admin', 'User', 'User']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)
  
user = 'DB2INST1'
password = 'db2inst1'
profile = 'HISTORY'

for row in range(0, len(userListDF)):
    Type = userListDF['Type'].iloc[row]
    newUser = userListDF['newUser'].iloc[row]
    
    script = 'GRANT DBADM, CREATETAB, BINDADD, CONNECT, CREATE_NOT_FENCED, IMPLICIT_SCHEMA, LOAD ON DATABASE TO USER "'+newUser+'";'
    if userListDF['Type'].iloc[row] == 'Admin' :
        script = script + 'GRANT ROLE "CONSOLE_ADM" TO USER "'+newUser+'";'
    else : 
        script = script + 'GRANT ROLE "CONSOLE_USR" TO USER "'+newUser+'";'        
    print('Adding User: '+newUser)
    displayResults(runSQL(profile, user, password, script))
print('done')

,newUser,Type
0,PETER,Admin
1,PAUL,User
2,MARY,User


Adding User: PETER
Statement: 0: GRANT DBADM, CREATETAB, BINDADD, CONNECT, CREATE_NOT_FENCED, IMPLICIT_SCHEMA, LOAD ON DATABASE TO USER "PETER"
No errors. Row Affected: 0

Statement: 1: GRANT ROLE "CONSOLE_ADM" TO USER "PETER"
No errors. Row Affected: 0

Adding User: PAUL
Statement: 0: GRANT DBADM, CREATETAB, BINDADD, CONNECT, CREATE_NOT_FENCED, IMPLICIT_SCHEMA, LOAD ON DATABASE TO USER "PAUL"
No errors. Row Affected: 0

Statement: 1: GRANT ROLE "CONSOLE_USR" TO USER "PAUL"
No errors. Row Affected: 0

Adding User: MARY
Statement: 0: GRANT DBADM, CREATETAB, BINDADD, CONNECT, CREATE_NOT_FENCED, IMPLICIT_SCHEMA, LOAD ON DATABASE TO USER "MARY"
No errors. Row Affected: 0

Statement: 1: GRANT ROLE "CONSOLE_USR" TO USER "MARY"
No errors. Row Affected: 0

done


### Switch Authentication Method in the Console
Next that we can users that are assigned to the correct role, we can switch from using the single default user to using Repository Database authentication for multiple users. You can do this through the Db2 Data Management Console and navigating to the **Repository Setting** page or by running the next cell below. Pick one of the two methods below to get to the Repository Settings page. Then follow the 10 steps to change the authentication method.

Use the full console to switch to Repository Database Authentication:
1. Click the link http://localhost:11080/console
2. Click the gear icon at the top right of the page
3. Select **Repository Setting**

Use the embedded page from the console to switch to Repostory Database Authentication
1. Run the cell below

Now configure the console to use the Repository database to authenticate users. 
1. Select **Repository** from the list of available Authentiation Types.
2. Scroll down and select **Db2 Roles** from the list of User role matting methods
3. Enter CONSOLE_ADM in the Admin roles field
4. Enter CONSOLE_USR in the User roles field
5. Click Next
6. Enter **db2inst1** in the Test user ID field
7. Enter **db2inst1** in the Test user password field
8. Click **Test**. You should see confirmation that the test succeeded
9. Select **Save** at the bottom right of the page. A confirmation dialog appears.
10. Click **Yes**. You should now see that Repository authentication is set and enabled successfully.


In [15]:
IFrame(database+'/console/?mode=compact#settings/authentication', width=1400, height=500)

### Test the new users
To test the new user and authentication method you can log out and into the console and you can run the cell below. 

1. Open the console http://localhost:11080/console
2. Click the user icon at the very top right of the screen
3. Select **sign out** a confirmation dialog appears
4. Select **Yes**
5. Enter **Peter** in the use field
6. Enter **dmc2020pot** in the password field

This next cell runs a simple query through the console service using one of the new users. 

In [29]:
sql = 'select TABSCHEMA, TABNAME from syscat.tables'
displayResults(runSQL('HISTORY', 'Peter', 'dmc2020pot', sql))

Statement: 0: select TABSCHEMA, TABNAME from syscat.tables
    TABSCHEMA                         TABNAME
0    DB2INST1                     DEPARTMENTS
1    DB2INST1                       EMPLOYEES
2    DB2INST1                       PERSONNEL
3  IBMCONSOLE                ACCESS_PLAN_DATA
4  IBMCONSOLE             ADMIN_EDITOR_RESULT
5  IBMCONSOLE            ADMIN_EDITOR_SCRIPTS
6  IBMCONSOLE     ADMIN_EDITOR_SCRIPTS_OPTION
7  IBMCONSOLE  ADMIN_EDITOR_SCRIPTS_SUBSCRIBE
8  IBMCONSOLE                           ALERT
9  IBMCONSOLE                ALERT_PROPERTIES



## Manage Console User Priviledges
In the console you can control which users can access specific database connections. These are divided into Owners, who can grant privleges, and users who can only use the connection.

### Get User Connection Profile Privileges
The next cell lets you control exactly which users can have specific databases. You can also access this through the console **Users and Privileges** settings page in the main console. 

In [30]:
IFrame(database+'/console/?mode=compact#settings/users', width=1400, height=300)

If you have a lot to users and databases to manage it may be useful to script these changes. The next cell include and example of how to retrieve the current privileges through the console APIs. 

In [31]:
# Get User Connection Profile Privleges
r = databaseAPI.getUserPriviledges()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

,profileName,OWNER,USER
0,SAMPLE,"[owner, db2inst1, user]","[db2pot, dmcusr]"
1,HISTORY,"[owner, db2inst1, user]","[db2pot, dmcusr]"


### Change User Connection Profile Privileges
If you have a lot of changes to make at once, the following cells are examples of how to script changes to multiple user and database connection priviledges. The first two cells in this section define reusable routines that let you add and revoke privileges with a single call. 

In [33]:
# Change User Connection Profile Privileges
def addProfilePrivileges(profile, name, userType) :

    if userType == 'user' : 
        r = databaseAPI.assignUserPrivileges(profile, name)
    else: 
        r = databaseAPI.assignOwnerPrivileges(profile, name)

    if (databaseAPI.getStatusCode(r)==201):
        print(name+' added to: '+profile+" as a new "+userType+".")
    else:
        print(databaseAPI.getStatusCode(r)) 
    
print('Created routine to add profile privileges')

Created routine to add profile privileges


In [34]:
# Revoke User Connection Profile Privileges
def revokeProfilePrivileges(profile, name) :

    r = databaseAPI.revokeProfilePrivileges(profile, name)

    if (databaseAPI.getStatusCode(r)==201):
        print(name+' privilege revoked from: '+profile)
    else:
        print(databaseAPI.getStatusCode(r)) 
    
print('Created routine to revoke profile privileges')

Created routine to revoke profile privileges


In [35]:
addProfilePrivileges('HISTORY', 'PAUL', 'owner')

PAUL added to: HISTORY as a new owner.


In [36]:
revokeProfilePrivileges('HISTORY', 'PAUL')

PAUL privilege revoked from: HISTORY


### Making Mutliple Changes at once
These next two examples, show how to process an entire list of changes through a simple loop. Both to grant and revoke user privileges. 

In [37]:
userList = {'userName':['PETER', 'PAUL', 'MARY', 'PETER', 'PAUL', 'MARY'], 'Profile':['HISTORY', 'HISTORY', 'HISTORY', 'SAMPLE', 'SAMPLE','SAMPLE'],'Type':['owner', 'user', 'user', 'owner', 'user', 'user']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)

for row in range(0, len(userListDF)):
    profile = userListDF['Profile'].iloc[row]
    userType = userListDF['Type'].iloc[row]
    name = userListDF['userName'].iloc[row]   
    addProfilePrivileges(profile, name, userType)
print('done')

,userName,Profile,Type
0,PETER,HISTORY,owner
1,PAUL,HISTORY,user
2,MARY,HISTORY,user
3,PETER,SAMPLE,owner
4,PAUL,SAMPLE,user
5,MARY,SAMPLE,user


PETER added to: HISTORY as a new owner.
PAUL added to: HISTORY as a new user.
MARY added to: HISTORY as a new user.
PETER added to: SAMPLE as a new owner.
PAUL added to: SAMPLE as a new user.
MARY added to: SAMPLE as a new user.
done


In [38]:
userList = {'userName':['PETER', 'PAUL', 'MARY', 'PETER', 'PAUL', 'MARY'], 'Profile':['HISTORY', 'HISTORY', 'HISTORY', 'SAMPLE', 'SAMPLE','SAMPLE']} 
userListDF = pd.DataFrame(userList) 
display(userListDF)

for row in range(0, len(userListDF)):
    profile = userListDF['Profile'].iloc[row]
    name = userListDF['userName'].iloc[row]   
    revokeProfilePrivileges(profile, name)
print('done')


,userName,Profile
0,PETER,HISTORY
1,PAUL,HISTORY
2,MARY,HISTORY
3,PETER,SAMPLE
4,PAUL,SAMPLE
5,MARY,SAMPLE


PETER privilege revoked from: HISTORY
PAUL privilege revoked from: HISTORY
MARY privilege revoked from: HISTORY
PETER privilege revoked from: SAMPLE
PAUL privilege revoked from: SAMPLE
MARY privilege revoked from: SAMPLE
done


### Adding Connection Profiles

Let's start by checking the list of current database profiles available in the console.

In [ ]:
IFrame(database+'/console/?mode=compact#connection/manager', width=1400, height=300)

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConsoleProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

In [ ]:
# Export Console Connection Profiles
r = databaseAPI.getExportConsoleProfiles()

if (databaseAPI.getStatusCode(r)==200):
    with open('connectionProfile.csv','wb') as file:
        file.write(r.content)
else:
    print(databaseAPI.getStatusCode(r))  

In [ ]:
# Import Console Connection Profile
df = pd.read_csv("connectionProfile.csv") 
df = df.drop(df.index[1]) #Drop the second row
df = df.iloc[:, :-1] # Drop the last column
display(df)

In [ ]:
# Extract Connection Credentials


### Re-enable the original setup administrator account

The setup administrator account is automatically disabled when the administrator account is configured the authentication settings. However, during emergency situations (for example, when the configured LDAP server is down, or the entries of administrator accounts are removed or renamed) when none of the administrator account is available to log into the console, it becomes necessary to re-enable the setup administrator account.

To re-enable setup administrator account:
1. Click the **Files** icon at the bottom left of the screen
2. Select the **Home** directory
3. Select the **dmc** directory
3. Select the **Config** directory
4. Select the **dewebserver_override.properties** file
5. Hit the **Enter** key
6. In the file editor change the property value named **auth** to **superAdmin** (auth=superAdmin, no spaces before or after)
7. Click **Save** at the top right of the editor screen
8. If necessary log out of the IBM Db2 Data Management Console and log in with the setup administrator account.
    Userid: db2inst1
    Password: db2inst1
9. Navigate to Authentication Setting page and configure the authentication settings. You should see that the settings have returned to the default configuration.

### Encrypt a password

Anytime that the Db2 Data Management Console stores a password, it encrypts it using a routine included with the console. You can access this yourself through the command line. 

To encrypt a password:
1. Click the **Terminal** icon at the bottom left of the screen
2. Go to the directory where the console is installed by running the following command
    cd dmc
2. Enter the following command where the NewPassword id **db2inst1**
```
dsutil/bin/crypt.sh db2inst1
```    
3. Copy the encrypted password by highlighting it with your mouse, right click and select Copy.

### Create a new Db2 Database

To create a new Db2 database you need to access the Db2 Terminal:

1. Click the **Terminal** icon at the bottom left of the screen
2. Switch to the DB2INST1 USER by running the following command
```
su - db2inst1
```
3. Enter the password **db2inst1**
4. Create a database with the following command where **dbone** is the name of the new database
```
db2 create database dbone
```

### Create a new Database Connection Profile through the User Interface

In [ ]:
IFrame(database+'/console/?mode=compact#connection/manager/add', width=1400, height=500)

### List the current Connection Profiles

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConsoleProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Check Connection Information without Creating a new Profile

In [ ]:
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
r = databaseAPI.postTestConnection(dbName, port, host, userid, password)
if (databaseAPI.getStatusCode(r)==200):
    print(databaseAPI.getJSON(r))
else:
    print(databaseAPI.getStatusCode(r)) 

### Create a new Database Connection Profile

In [ ]:
connectionName = 'dbOneNew3'
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
comment = 'new connection profile test'
r = databaseAPI.postConnectionProfile(connectionName, dbName, port, host, userid, password, comment)
if (databaseAPI.getStatusCode(r)==201):
    print("Created connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

### List the current Connection Profiles

In [ ]:
# Get Console Connection Profiles
r = databaseAPI.getConsoleProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    display(pd.DataFrame(json_normalize(json)))
else:
    print(databaseAPI.getStatusCode(r))      

### Delete a connection profile

In [ ]:
connectionName = 'dbOneNew3'

r = databaseAPI.deleteConnectionProfile(connectionName)
if (databaseAPI.getStatusCode(r)==200):
    print("Deleted connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

### Update an Existing Database Connection Profile

In [ ]:
connectionName = 'dbOneNew2'
dbName = 'dbone'
port = '50000'
host = 'localhost'
userid = 'db2inst1'
password = 'db2inst1'
comment = 'Updated Comment'
r = databaseAPI.putConnectionProfileUpdate(connectionName, dbName, port, host, userid, password, comment)
if (databaseAPI.getStatusCode(r)==200):
    print("Updated connection profile "+connectionName)
else:
    print(databaseAPI.getStatusCode(r)) 

### List Monitor Profiles

In [ ]:
IFrame(database+'/console/?mode=compact#monitorprofile', width=1400, height=300)

In [ ]:
r = databaseAPI.getMonitoringProfiles()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    profileList = pd.DataFrame(json_normalize(json['resources']))
    display(profileList)
else:
    print(databaseAPI.getStatusCode(r)) 

### Get a Monitor Profile

In [ ]:
profileName = "Test"
profileID = str(databaseAPI.getProfileIndex(profileName))
r = databaseAPI.getMonitoringProfile(profileID)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
    print(json['base_info'])
    print(json['monitor_config']['common_settings'])
else:
    print(databaseAPI.getStatusCode(r))    

### Change a Monitoring Profile

In [ ]:
profileName = "Test"

# Get the existing json for the monitoring profile
profileID = str(databaseAPI.getProfileIndex(profileName))
r = databaseAPI.getMonitoringProfile(profileID)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r) 
else:
    print(databaseAPI.getStatusCode(r))    

# Change the json that describes the list of databases assigned to this monitoring profile
json['base_info']['assign_database_list'] = "dbOneNew2,Fred"
json['monitor_config']['common_settings']['collect_data_every'] = 10  

# Put the updated profile back
r = databaseAPI.putMonitoringProfile(profileID, json)
if (databaseAPI.getStatusCode(r)==200):
    print(profileName+' Monitoring Profile Updated')
    print(json['base_info'])
    print(json['monitor_config']['common_settings'])
else:
    print(databaseAPI.getStatusCode(r)) 

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]